## Import Library

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from scipy import stats

## Import CSV

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/online_retail_II.csv", header = 0)
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


## Data Cleansing

In [ ]:
df_resik = df.copy()

#create date columns
df_resik['order_date'] = df_resik['InvoiceDate'].astype('datetime64[ns]')

# Convert datetime
df_resik['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# Create month column
df_resik['year_month'] = df_resik['order_date'].dt.to_period('M')

# Remove all customer id
df_resik = df_resik[~df_resik['Customer ID'].isna()]

# Remove all description
df_resik = df_resik[~df_resik['Description'].isna()]

# Make all item in description column with no capital
df_resik['Description'] = df_resik['Description'].str.lower()

# Remove all name in the row, that name p, w and post or special characters
df_resik = df_resik[~df_resik['StockCode'].str.match('^\d+$', case=False, na=False)]

# Adding order status
df_resik['order_status'] = np.where(df_resik['Invoice'].str[:1]=='C', 'Cancelled', 'Delivered')

# change negatif to positif in quantity
df_resik['Quantity'] = df_resik['Quantity'].abs()

# Remove row that there's negative
df_resik = df_resik[df_resik['Quantity'] > 0]

# Create new column by name amount
df_resik['Amount'] = df_resik['Quantity'] * df_resik['Price']

# Get the most common in description for every stockcode
most_freq_product_name = df_resik.groupby(['StockCode', 'Description'], as_index=False) \
  .agg(order_cnt=('Invoice', 'nunique')).sort_values(['StockCode', 'order_cnt'],ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('StockCode')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt', 'rank'])

df_resik = df_resik.merge(
    most_freq_product_name.rename(columns={'Description':'most_freq_product_name'}),
    on='StockCode',
    how='left'
)

df_resik['Description'] = df_resik['most_freq_product_name']
df_resik = df_resik.drop(columns=['most_freq_product_name'])

# change customer ID to String
df_resik['Customer ID'] = df_resik['Customer ID'].astype(str)

# outlier remove
df_resik = df_resik[(np.abs(stats.zscore(df_resik[['Quantity', 'Amount']])) < 3).all(axis=1)]
df_resik = df_resik.reset_index(drop=True)
df_resik

/tmp/ipython-input-4-1964579976.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resik['order_status'] = np.where(df_resik['Invoice'].str[:1]=='C', 'Cancelled', 'Delivered')
/tmp/ipython-input-4-1964579976.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resik['Quantity'] = df_resik['Quantity'].abs()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,order_date,year_month,order_status,Amount
0,489434,79323P,pink cherry lights,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01 07:45:00,2009-12,Delivered,81.00
1,489434,79323W,white cherry lights,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01 07:45:00,2009-12,Delivered,81.00
2,489436,48173C,doormat black flock,10,2009-12-01 09:06:00,5.95,13078.0,United Kingdom,2009-12-01 09:06:00,2009-12,Delivered,59.50
3,489436,35004B,set of 3 black flying ducks,12,2009-12-01 09:06:00,4.65,13078.0,United Kingdom,2009-12-01 09:06:00,2009-12,Delivered,55.80
4,489436,84596F,small marshmallows pink bowl,8,2009-12-01 09:06:00,1.25,13078.0,United Kingdom,2009-12-01 09:06:00,2009-12,Delivered,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...
93933,581579,85099C,jumbo bag baroque black white,10,2011-12-09 12:19:00,1.79,17581.0,United Kingdom,2011-12-09 12:19:00,2011-12,Delivered,17.90
93934,581580,84993A,75 green petit four cases,2,2011-12-09 12:20:00,0.42,12748.0,United Kingdom,2011-12-09 12:20:00,2011-12,Delivered,0.84
93935,581580,85049A,traditional christmas ribbons,1,2011-12-09 12:20:00,1.25,12748.0,United Kingdom,2011-12-09 12:20:00,2011-12,Delivered,1.25
93936,581580,85049E,scandinavian reds ribbons,2,2011-12-09 12:20:00,1.25,12748.0,United Kingdom,2011-12-09 12:20:00,2011-12,Delivered,2.50


In [ ]:
df_resik.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93938 entries, 0 to 93937
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Invoice       93938 non-null  object        
 1   StockCode     93938 non-null  object        
 2   Description   93938 non-null  object        
 3   Quantity      93938 non-null  int64         
 4   InvoiceDate   93938 non-null  datetime64[ns]
 5   Price         93938 non-null  float64       
 6   Customer ID   93938 non-null  object        
 7   Country       93938 non-null  object        
 8   order_date    93938 non-null  datetime64[ns]
 9   year_month    93938 non-null  period[M]     
 10  order_status  93938 non-null  object        
 11  Amount        93938 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(6), period[M](1)
memory usage: 8.6+ MB


## Create RFM Segmentation

Aggregate transaction data into a summary form of total transactions (orders), total order value (order value), last order date for each user

In [ ]:
df_user_month = df_resik.groupby('Customer ID', as_index=False) \
  .agg(order_cnt=('Invoice', 'nunique'), max_order_date=('order_date', 'max'), total_order_value=('Amount', 'sum'))
df_user_month

,Customer ID,order_cnt,max_order_date,total_order_value
0,12346.0,15,2010-10-04 16:33:00,666.38
1,12347.0,8,2011-12-07 15:52:00,1013.00
2,12348.0,5,2011-09-25 13:13:00,361.00
3,12349.0,4,2011-11-21 09:51:00,1109.63
4,12350.0,1,2011-02-02 16:01:00,76.40
...,...,...,...,...
5019,18283.0,17,2011-12-06 12:02:00,209.20
5020,18284.0,2,2010-10-06 12:31:00,95.40
5021,18285.0,1,2010-02-17 10:24:00,65.90
5022,18286.0,3,2010-08-20 11:57:00,456.13


## create colom calculate day since last order

In [ ]:
today = df_resik['order_date'].max()
df_user_month['day_last_order'] = (today - df_user_month['max_order_date']).dt.days
df_user_month

,Customer ID,order_cnt,max_order_date,total_order_value,day_last_order
0,12346.0,15,2010-10-04 16:33:00,666.38,430
1,12347.0,8,2011-12-07 15:52:00,1013.00,1
2,12348.0,5,2011-09-25 13:13:00,361.00,74
3,12349.0,4,2011-11-21 09:51:00,1109.63,18
4,12350.0,1,2011-02-02 16:01:00,76.40,309
...,...,...,...,...,...
5019,18283.0,17,2011-12-06 12:02:00,209.20,3
5020,18284.0,2,2010-10-06 12:31:00,95.40,429
5021,18285.0,1,2010-02-17 10:24:00,65.90,660
5022,18286.0,3,2010-08-20 11:57:00,456.13,476


In [ ]:
df_user_month.describe()

,order_cnt,max_order_date,total_order_value,day_last_order
count,5024.000000,5024,5024.000000,5024.000000
mean,5.176154,2011-04-16 09:42:39.745222912,414.440272,236.602110
min,1.000000,2009-12-01 09:55:00,0.000000,0.000000
25%,1.000000,2010-10-24 15:50:00,40.060000,36.000000
50%,3.000000,2011-06-28 11:56:30,110.430000,164.000000
75%,6.000000,2011-11-02 14:07:15,315.077500,410.000000
max,359.000000,2011-12-09 12:50:00,49158.710000,738.000000
std,10.937273,NaN,1575.957547,218.274468


## Create a binning of the number of days since the last order consisting of 5 bins with the boundaries being min, P20, P40, P60, P80, max and label 1 to 5 from highest to lowest bin as the current score

In [ ]:
df_user_month['recency_score'] = pd.cut(df_user_month['day_last_order'],
                                        bins=[df_user_month['day_last_order'].min(),
                                              np.percentile(df_user_month['day_last_order'], 20),
                                              np.percentile(df_user_month['day_last_order'], 40),
                                              np.percentile(df_user_month['day_last_order'], 60),
                                              np.percentile(df_user_month['day_last_order'], 80),
                                              df_user_month['day_last_order'].max()],
                                        labels= [5, 4, 3, 2, 1],
                                        include_lowest=True).astype(str)
df_user_month

,Customer ID,order_cnt,max_order_date,total_order_value,day_last_order,recency_score
0,12346.0,15,2010-10-04 16:33:00,666.38,430,2
1,12347.0,8,2011-12-07 15:52:00,1013.00,1,5
2,12348.0,5,2011-09-25 13:13:00,361.00,74,4
3,12349.0,4,2011-11-21 09:51:00,1109.63,18,5
4,12350.0,1,2011-02-02 16:01:00,76.40,309,2
...,...,...,...,...,...,...
5019,18283.0,17,2011-12-06 12:02:00,209.20,3,5
5020,18284.0,2,2010-10-06 12:31:00,95.40,429,2
5021,18285.0,1,2010-02-17 10:24:00,65.90,660,1
5022,18286.0,3,2010-08-20 11:57:00,456.13,476,1


In [ ]:
df_user_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5024 entries, 0 to 5023
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Customer ID        5024 non-null   object        
 1   order_cnt          5024 non-null   int64         
 2   max_order_date     5024 non-null   datetime64[ns]
 3   total_order_value  5024 non-null   float64       
 4   day_last_order     5024 non-null   int64         
 5   recency_score      5024 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 235.6+ KB


 ## Create a binning of total transactions (orders) consisting of 5 bins with the boundaries being min, P20, P40, P60, P80, max and label them 1 to 5 from lowest to highest bin as frequency score

In [ ]:
df_user_month['frequency_score'] = pd.cut(df_user_month['order_cnt'],
                                          bins=[0,
                                                np.percentile(df_user_month['order_cnt'], 20),
                                                np.percentile(df_user_month['order_cnt'], 40),
                                                np.percentile(df_user_month['order_cnt'], 60),
                                                np.percentile(df_user_month['order_cnt'], 80),
                                                df_user_month['order_cnt'].max()],
                                          labels=[1, 2, 3, 4 ,5],
                                          include_lowest=True).astype(int)
df_user_month

,Customer ID,order_cnt,max_order_date,total_order_value,day_last_order,recency_score,frequency_score
0,12346.0,15,2010-10-04 16:33:00,666.38,430,2,5
1,12347.0,8,2011-12-07 15:52:00,1013.00,1,5,5
2,12348.0,5,2011-09-25 13:13:00,361.00,74,4,4
3,12349.0,4,2011-11-21 09:51:00,1109.63,18,5,4
4,12350.0,1,2011-02-02 16:01:00,76.40,309,2,1
...,...,...,...,...,...,...,...
5019,18283.0,17,2011-12-06 12:02:00,209.20,3,5,5
5020,18284.0,2,2010-10-06 12:31:00,95.40,429,2,2
5021,18285.0,1,2010-02-17 10:24:00,65.90,660,1,1
5022,18286.0,3,2010-08-20 11:57:00,456.13,476,1,3


##  Create a binning of total transactions (monetary_score) consisting of 5 bins with the boundaries being min, P20, P40, P60, P80, max and label them 1 to 5 from lowest to highest bin as frequency score

In [ ]:
df_user_month['monetary_score'] = pd.cut(df_user_month['total_order_value'],
                                         bins=[df_user_month['total_order_value'].min(),
                                               np.percentile(df_user_month['total_order_value'], 20),
                                               np.percentile(df_user_month['total_order_value'], 40),
                                               np.percentile(df_user_month['total_order_value'], 60),
                                               np.percentile(df_user_month['total_order_value'], 80),
                                               df_user_month['total_order_value'].max()],
                                         labels=[1, 2, 3, 4, 5],
                                         include_lowest=True).astype(int)
df_user_month

,Customer ID,order_cnt,max_order_date,total_order_value,day_last_order,recency_score,frequency_score,monetary_score
0,12346.0,15,2010-10-04 16:33:00,666.38,430,2,5,5
1,12347.0,8,2011-12-07 15:52:00,1013.00,1,5,5,5
2,12348.0,5,2011-09-25 13:13:00,361.00,74,4,4,4
3,12349.0,4,2011-11-21 09:51:00,1109.63,18,5,4,5
4,12350.0,1,2011-02-02 16:01:00,76.40,309,2,1,3
...,...,...,...,...,...,...,...,...
5019,18283.0,17,2011-12-06 12:02:00,209.20,3,5,5,4
5020,18284.0,2,2010-10-06 12:31:00,95.40,429,2,2,3
5021,18285.0,1,2010-02-17 10:24:00,65.90,660,1,1,2
5022,18286.0,3,2010-08-20 11:57:00,456.13,476,1,3,5


## Create a segment name column based on recency and frequency scores

In [ ]:
df_user_month['recency_score'] = df_user_month['recency_score'].astype(int)
df_user_month['frequency_score'] = df_user_month['frequency_score'].astype(int)
df_user_month['monetary_score'] = df_user_month['monetary_score'].astype(int)

df_user_month['segment_rfm'] = np.select(
    [(df_user_month['recency_score'] == 5) & (df_user_month['frequency_score'] >= 4) & (df_user_month['monetary_score'] >= 4),
      (df_user_month['recency_score'].between(3, 4)) & (df_user_month['frequency_score'] >= 4) & (df_user_month['monetary_score'] >= 3),
      (df_user_month['recency_score'] >= 4) & (df_user_month['frequency_score'].between(2, 3)) & (df_user_month['monetary_score'].between(3, 5)),
      (df_user_month['recency_score'] <= 2) & (df_user_month['frequency_score'] == 5) & (df_user_month['monetary_score'] >= 4),
      (df_user_month['recency_score'] == 3) & (df_user_month['frequency_score'] == 3) & (df_user_month['monetary_score'] == 3),
      (df_user_month['recency_score'] == 5) & (df_user_month['frequency_score'] == 1) & (df_user_month['monetary_score'] >= 4),
      (df_user_month['recency_score'] == 4) & (df_user_month['frequency_score'] == 1) & (df_user_month['monetary_score'].between(2,3)),
      (df_user_month['recency_score'] <= 2) & (df_user_month['frequency_score'].between(3, 4)) & (df_user_month['monetary_score'].between(2,4)),
      (df_user_month['recency_score'] == 3) & (df_user_month['frequency_score'] <= 2) & (df_user_month['monetary_score'] <= 3),
      (df_user_month['recency_score'] <= 2) & (df_user_month['frequency_score'] <= 2) & (df_user_month['monetary_score'] <= 2)],
    ['01-Champion',
      '02-Loyal Customers',
      '03-Potential Loyalists',
      "04-Can't Lose Them",
      '05-Need Attention',
      '06-New Customers',
      '07-Promising',
      '08-At Risk',
      '09-About to Sleep',
      '10-Hibernating'],
    default='Other'
)
df_user_month

,Customer ID,order_cnt,max_order_date,total_order_value,day_last_order,recency_score,frequency_score,monetary_score,segment,segment_rfm
0,12346.0,15,2010-10-04 16:33:00,666.38,430,2,5,5,04-Can't Lose Them,04-Can't Lose Them
1,12347.0,8,2011-12-07 15:52:00,1013.00,1,5,5,5,01-Champion,01-Champion
2,12348.0,5,2011-09-25 13:13:00,361.00,74,4,4,4,02-Loyal Customers,02-Loyal Customers
3,12349.0,4,2011-11-21 09:51:00,1109.63,18,5,4,5,01-Champion,01-Champion
4,12350.0,1,2011-02-02 16:01:00,76.40,309,2,1,3,10-Hibernating,Other
...,...,...,...,...,...,...,...,...,...,...
5019,18283.0,17,2011-12-06 12:02:00,209.20,3,5,5,4,01-Champion,01-Champion
5020,18284.0,2,2010-10-06 12:31:00,95.40,429,2,2,3,10-Hibernating,Other
5021,18285.0,1,2010-02-17 10:24:00,65.90,660,1,1,2,10-Hibernating,10-Hibernating
5022,18286.0,3,2010-08-20 11:57:00,456.13,476,1,3,5,08-At Risk,Other


## Display a summary of RFM segmentation (point 8) in the form of number of users, average and median of total orders, total order value, and number of days since the last order

In [ ]:
summarry_all = pd.pivot_table(df_user_month, index='segment_rfm',
                              values=['Customer ID', 'day_last_order', 'order_cnt', 'total_order_value'],
                              aggfunc={'Customer ID' : pd.Series.nunique,
                                       'day_last_order' : [np.mean, np.sum],
                                       'order_cnt' : [np.mean, np.sum],
                                       'total_order_value' : [np.mean, np.sum]})
summarry_all['pct_unique'] = (summarry_all['Customer ID'] / summarry_all['Customer ID'].sum() * 100).round(1)
summarry_all

/tmp/ipython-input-23-4234339553.py:1: FutureWarning: The provided callable <function mean at 0x78bbdb3320c0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  summarry_all = pd.pivot_table(df_user_month, index='segment_rfm',
/tmp/ipython-input-23-4234339553.py:1: FutureWarning: The provided callable <function sum at 0x78bbdb330fe0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  summarry_all = pd.pivot_table(df_user_month, index='segment_rfm',


Customer ID day_last_order          order_cnt         \
                           nunique           mean     sum       mean    sum   
segment_rfm                                                                   
01-Champion                    565      10.973451    6200  17.892035  10109   
02-Loyal Customers             916      99.241266   90905   8.713974   7982   
03-Potential Loyalists         270      37.996296   10259   2.544444    687   
04-Can't Lose Them              55     380.072727   20904  14.127273    777   
05-Need Attention               51     190.627451    9722   3.000000    153   
06-New Customers                 3      14.666667      44   1.000000      3   
07-Promising                    64      53.859375    3447   1.000000     64   
08-At Risk                     362     410.149171  148474   3.883978   1406   
09-About to Sleep              398     177.979899   70836   1.339196    533   
10-Hibernating                1035     498.362319  515805   1.192271   1234   
Other                         1305     239.151724  312093   2.342529   3057   

                       total_order_value             pct_unique  
                                    mean         sum             
segment_rfm                                                      
01-Champion                  1717.925069  970627.664       11.2  
02-Loyal Customers            707.331860  647915.984       18.2  
03-Potential Loyalists        188.920519   51008.540        5.4  
04-Can't Lose Them           1207.884382   66433.641        1.1  
05-Need Attention             112.758824    5750.700        1.0  
06-New Customers              320.066667     960.200        0.1  
07-Promising                   66.748906    4271.930        1.3  
08-At Risk                    171.870608   62217.160        7.2  
09-About to Sleep              47.778623   19015.892        7.9  
10-Hibernating                 32.476070   33612.732       20.6  
Other                         168.837920  220333.485       26.0